<a href="https://colab.research.google.com/github/41371109h/114-1/blob/main/hw6%EF%BC%BF%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install gspread pandas gradio -q

In [4]:
import gspread
import pandas as pd
import gradio as gr
from datetime import datetime
from typing import List, Dict, Tuple

# ==============================================================================
# 0. 配置
# ==============================================================================

# ⚠️ 請確保您的金鑰檔案名為 'gspread_key.json' 且與腳本在同一目錄
CREDENTIALS_FILE = 'gspread_key.json'

# 課表所在的分頁名稱 (通常為 '課表' 或 'Sheet1')
SCHEDULE_SHEET_NAME = '課表1'

# 欲回寫的目標分頁名稱
FOCUS_SHEET_NAME = '本週重點'

# 星期名稱映射
DAY_MAPPING = {
    "一": "MON", "二": "TUE", "三": "WED", "四": "THU", "五": "FRI", "六": "SAT", "日": "SUN",
    "MON": "星期一", "TUE": "星期二", "WED": "星期三", "THU": "星期四", "FRI": "星期五",
    "SAT": "星期六", "SUN": "星期日"
}

# 輔助課程內容判斷 (用於 AI 點子)
COURSE_TAGS = {
    '實作/模型': ['木工製造', '機械製造', '基礎模型製作', '基礎設計'],
    '戶外/體育': ['體育(網球初級)'],
    '理論/閱讀': ['科技系統與社會發展', '文明的探索', '色彩學', '課程發展與設計', '教師專業發展'],
    '程式/電腦': ['程式語言'],
    '教室': ['TC', 'TB', '誠', '設計系', '401'],
}

# ==============================================================================
# 1. Google Sheets 讀寫函式 (修改為使用 URL 開啟)
# ==============================================================================

def get_google_sheet_data(sheet_url: str, sheet_name: str) -> pd.DataFrame:
    """從 Google Sheets 讀取指定分頁的資料並轉換為 DataFrame (使用 URL)。"""
    try:
        gc = gspread.service_account(filename=CREDENTIALS_FILE)

        # 🚨 使用 open_by_url 根據連結開啟試算表
        sh = gc.open_by_url(sheet_url)

        worksheet = sh.worksheet(sheet_name)
        data = worksheet.get_all_values()

        if not data: return pd.DataFrame()

        # 清理並使用第一行作為標題
        headers = [h.strip().split('\n')[0] for h in data[0]]
        df = pd.DataFrame(data[1:], columns=headers)
        df.columns = df.columns.str.replace(r'\n.*', '', regex=True).str.strip()

        return df

    except gspread.exceptions.WorksheetNotFound:
        return pd.DataFrame()
    except Exception as e:
        print(f"Google Sheets 讀取錯誤，請檢查 URL 和分享設定：{e}")
        return pd.DataFrame()

def write_to_focus_sheet(sheet_url: str, day_name: str, reminder: str, daily_schedule: pd.DataFrame) -> str:
    """將分析結果回寫到 '本週重點' 分頁 (使用 URL 開啟)。"""
    try:
        gc = gspread.service_account(filename=CREDENTIALS_FILE)
        sh = gc.open_by_url(sheet_url) # 🚨 使用 open_by_url

        try:
            worksheet = sh.worksheet(FOCUS_SHEET_NAME)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = sh.add_worksheet(title=FOCUS_SHEET_NAME, rows="100", cols="20")

        date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # 準備要回寫的內容
        schedule_data = daily_schedule.to_numpy().tolist()
        schedule_header = daily_schedule.columns.tolist()

        output_data = [
            [f"--- {day_name} 課程重點 ({date_str}) ---"],
            [],
            ["💡 AI 行前提醒:", reminder],
            [],
            ["當日詳細課表："],
            schedule_header
        ] + schedule_data

        worksheet.clear()
        worksheet.update('A1', output_data)

        return f"\n✅ 成功將 {day_name} 的課程重點與提醒回寫至 Google Sheets 的 '{FOCUS_SHEET_NAME}' 分頁。"

    except Exception as e:
        return f"\n回寫 Google Sheets 錯誤：{e}"


# ==============================================================================
# 2. AI 點子：生成行前提醒函式 (保持不變)
# ==============================================================================

def generate_daily_reminder(daily_schedule: pd.DataFrame) -> str:
    """根據當日課程生成一句「行前提醒」"""
    if daily_schedule.empty:
        return "今日無課程，是休息或預習的最佳時機！"

    courses = daily_schedule['課程名稱'].unique().tolist()
    reminders = []

    # 識別課程類型並生成提醒
    if any(any(tag in c for tag in COURSE_TAGS['實作/模型']) for c in courses): reminders.append("實作/模型課，務必記得攜帶所需材料或工具。")
    if any(any(tag in c for tag in COURSE_TAGS['戶外/體育']) for c in courses): reminders.append("有體育課，請準備好運動服和網球用品。")
    if any(any(tag in c for tag in COURSE_TAGS['程式/電腦']) for c in courses): reminders.append("程式語言課前請檢查筆電，並預習下一章節。")
    if any(any(tag in c for tag in COURSE_TAGS['理論/閱讀']) for c in courses): reminders.append("理論課程較多，請帶好筆記本和課本。")

    if len(reminders) == 0:
        return f"今日課程為：{', '.join(courses)}。請確認所有課本與文具已備妥。"

    return f"【行前提醒】{', '.join(reminders)}。"


# ==============================================================================
# 3. Gradio 主控函式 (接受 URL 輸入)
# ==============================================================================

def get_daily_schedule_and_reminders(sheet_url: str, target_day: str) -> str:
    """
    主控流程：讀取課表 -> 篩選 -> 生成提醒 -> 回寫 -> 輸出 Gradio 格式。
    """
    if not sheet_url or not target_day:
        return "https://docs.google.com/spreadsheets/d/1AMmd-Bv4QFLUrNY48YUeDl8I6Xts14Ew232uymWeHrg/edit?gid=14920929#gid=14920929。", ""

    day_abbr = DAY_MAPPING.get(target_day[-1].upper())
    day_full_name = DAY_MAPPING.get(day_abbr, target_day)


    # 1. 讀取課表
    df_schedule = get_google_sheet_data(sheet_url, SCHEDULE_SHEET_NAME)

    if df_schedule.empty:
        return f"### ❌ 讀取失敗\n\n無法取得課表資料。請檢查：\n1. Google Sheet 連結 (URL) 是否正確。\n2. 連結的分頁 ('{SCHEDULE_SHEET_NAME}') 是否存在。\n3. 您是否已分享給服務帳號。", ""

    # 2. 確定星期欄位名稱
    day_col_name = next((col for col in df_schedule.columns if day_abbr in col or day_full_name in col), None)

    if not day_col_name:
        return f"### ❌ 欄位匹配錯誤\n\n課表欄位中找不到 '{day_full_name}' 的對應欄位名稱。請檢查您的 Google Sheet 標題行。", ""

    # 3. 篩選當日課程
    daily_schedule = df_schedule[df_schedule[day_col_name].str.strip() != '']

    if daily_schedule.empty:
        reminder = generate_daily_reminder(pd.DataFrame())
        write_message = write_to_focus_sheet(sheet_url, day_full_name, reminder, pd.DataFrame())
        return f"### ✅ {day_full_name} 課程表\n\n**今日無課程安排**。\n\n💡 AI 行前提醒：{reminder}\n\n---\n{write_message}", ""


    # 4. 提取和清理課程名稱/地點
    output_schedule = daily_schedule[['節次', '時間', day_col_name]].copy()

    def parse_course(text):
        parts = [p.strip() for p in text.split('\n') if p.strip()]
        course = parts[0] if parts else ""
        location_candidates = [p for p in parts[1:] if any(tag in p for tag in COURSE_TAGS['教室'])]
        location = location_candidates[-1] if location_candidates else "未明確標示"
        return pd.Series([course, location])

    output_schedule[['課程名稱', '上課地點']] = output_schedule[day_col_name].apply(parse_course)
    output_schedule = output_schedule.drop(columns=[day_col_name])

    # 5. 生成 AI 行前提醒
    daily_reminder = generate_daily_reminder(output_schedule)

    # 6. 回寫「本週重點」分頁
    write_message = write_to_focus_sheet(sheet_url, day_full_name, daily_reminder, output_schedule)

    # 7. 格式化 Gradio 輸出
    markdown_output = f"### ✅ {day_full_name} 課程表 (讀取自 Google Sheets)\n\n"
    markdown_output += f"💡 **AI 行前提醒**：{daily_reminder}\n\n"
    markdown_output += "#### 詳細課程：\n"
    markdown_output += output_schedule.to_markdown(index=False)
    markdown_output += f"\n\n---\n{write_message}"

    return markdown_output


# ==============================================================================
# 4. Gradio 介面定義與啟動
# ==============================================================================

with gr.Blocks(title="Google Sheets 課表查詢與提醒") as demo:
    gr.Markdown("# 🗓️ Google Sheets 課表查詢與 AI 提醒工具")
    gr.Markdown("程式將從您的 Google Sheet 讀取課表，輸出當日課程，生成 AI 行前提醒，並將結果回寫至 '本週重點' 分頁。")
    gr.Markdown(f"**回寫目標分頁：** `{FOCUS_SHEET_NAME}`")
    gr.Markdown("---")

    with gr.Column(variant="panel"):
        gr.Markdown("#### 步驟一：輸入 Google Sheet 連結與分頁名稱")
        url_input = gr.Textbox(
            label="🔗 Google Sheet 連結 (URL)",
            placeholder="請貼上您的課表試算表完整連結，例如：https://docs.google.com/spreadsheets/d/.../edit",
            lines=1
        )
        gr.Markdown(f"💡 **提示:** 課表分頁名稱預設為：`{SCHEDULE_SHEET_NAME}`。")

    with gr.Row():
        day_input = gr.Dropdown(
            label="步驟二：選擇您想查詢的星期",
            choices=["星期一", "星期二", "星期三", "星期四", "星期五", "星期六"],
            value="星期一",
            interactive=True
        )
        search_button = gr.Button("🚀 查詢並回寫課表", variant="primary")

    output_display = gr.Markdown("📝 **結果將顯示於此**")

    search_button.click(
        fn=get_daily_schedule_and_reminders,
        inputs=[url_input, day_input],
        outputs=[output_display]
    )

if __name__ == '__main__':
    print("-----------------------------------------------------------------")
    print("  Gradio 應用程式啟動中...")
    print("-----------------------------------------------------------------\n")

    demo.launch(share=True)

-----------------------------------------------------------------
  Gradio 應用程式啟動中...
-----------------------------------------------------------------

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bcf128a44ecdc13e0f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
